In [10]:
import random
import numpy as np

In [2]:
dt = .001
tau = .1
noise_tau = .002
w1 = 1
w2 = 1
alpha = 15
beta = 1.1
dur = 10
sgm = .5
threshold = 70
S = 10

In [13]:
def ave_choice(r):

    R1_arr = np.zeros(10240)
    R2_arr = np.zeros(10240)
    G1_arr = np.zeros(10240)
    G2_arr = np.zeros(10240)
    D1_arr = np.zeros(10240)
    D2_arr = np.zeros(10240)

    noise_R1 = np.zeros(10240)
    noise_R2 = np.zeros(10240)
    noise_G1 = np.zeros(10240)
    noise_G2 = np.zeros(10240)
    noise_D1 = np.zeros(10240)
    noise_D2 = np.zeros(10240)

    choice_time = 0
    choice = 0

    for i in range(1/dt):
        V1 = 2 * S * r
        V2 = 2 * S - V1

    for ti in range(1, 10241):
        d_noiseR1 = (-noise_R1[ti-1] + random.randn() * np.sqrt(dt) * sgm) / noise_tau*dt
        d_noiseR2 = (-noise_R2[ti-1] + random.randn() * np.sqrt(dt) * sgm) / noise_tau*dt
        d_noiseG1 = (-noise_G1[ti-1] + random.randn() * np.sqrt(dt) * sgm) / noise_tau*dt
        d_noiseG2 = (-noise_G2[ti-1] + random.randn() * np.sqrt(dt) * sgm) / noise_tau*dt
        d_noiseD1 = (-noise_D1[ti-1] + random.randn() * np.sqrt(dt) * sgm) / noise_tau*dt
        d_noiseD2 = (-noise_D2[ti-1] + random.randn() * np.sqrt(dt) * sgm) / noise_tau*dt
        #d_noiseR1 = (-noiseR1(ti-1) + randn.*sqrt(dt).*sgm)/noise_tau*dt;

        #dR1 = (-R1(ti-1) + (alpha*R1(ti-1)+V1) / (1+G1(ti-1)))*dt/tau;
        d_R1 = (-R1_arr[ti-1] + (alpha * R1_arr[ti-1] + V1) / (1 + G1_arr[ti-1])) * dt/tau
        d_R2 = (-R2_arr[ti-1] + (alpha * R2_arr[ti-1] + V1) / (1 + G2_arr[ti-1])) * dt/tau
        #dG1 = (-G1(ti-1) + w1*R1(ti-1) + w2*R2(ti-1) - D1(ti-1))*dt/tau;
        d_G1 = (-G1_arr[ti-1] + w1*R1_arr[ti-1] + w2*R2_arr[ti-1] - D1_arr[ti-1]) * dt/tau
        d_G2 = (-G2_arr[ti-1] + w1*R1_arr[ti-1] + w2*R2_arr[ti-1] - D2_arr[ti-1]) * dt/tau
        #dD1 = (-D1(ti-1) + beta*R1(ti-1))*dt/tau;
        d_D1 = (-D1_arr[ti-1] + beta*R1_arr[ti-1]) *dt/tau
        d_D2 = (-D2_arr[ti-1] + beta*R2_arr[ti-1]) *dt/tau

        #noiseR1(ti) = noiseR1(ti-1) + d_noiseR1;
        noise_R1[ti] = noise_R1[ti-1] + d_noiseR1
        noise_R2[ti] = noise_R2[ti-1] + d_noiseR2
        noise_G1[ti] = noise_G1[ti-1] + d_noiseG1
        noise_G2[ti] = noise_G2[ti-1] + d_noiseG2
        noise_D1[ti] = noise_D1[ti-1] + d_noiseD1
        noise_D2[ti] = noise_D2[ti-1] + d_noiseD2

        #R1(ti) = max(R1(ti-1) + dR1 + noiseR1(ti), 0);
        R1_arr[ti] = max(R1_arr[ti-1] + d_R1 + noise_R1[ti], 0)
        R2_arr[ti] = max(R2_arr[ti-1] + d_R2 + noise_R2[ti], 0)
        G1_arr[ti] = max(G1_arr[ti-1] + d_G1 + noise_G1[ti], 0)
        G2_arr[ti] = max(G2_arr[ti-1] + d_G2 + noise_G2[ti], 0)
        D1_arr[ti] = max(D1_arr[ti-1] + d_D1 + noise_D1[ti], 0)
        D2_arr[ti] = max(D2_arr[ti-1] + d_D2 + noise_D2[ti], 0)

        if R1_arr[ti] > threshold:
            choice = choice + 1
            choice_time = choice_time + ti
            break
        elif R2_arr[ti] > threshold:
            choice_time = choice_time + ti
            break

    choice_time = choice_time / 10240
    choice = choice /10240

    return choice_time, choice